<a href="https://colab.research.google.com/github/DJCordhose/practical-llm/blob/main/sLLM-Eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Eval - small LLM as a judge

## Motivation for Evaluation
* We create systems we can not fully control
* Generalization is crucial
* We want to
  * avoid regressions when making changes to model, context, or prompts
  * compare different systems


### Regressions in Versions
![Regressions in Versions](https://raw.githubusercontent.com/DJCordhose/practical-llm/main/llm_regression.jpg "Regressions in Versions")


## Arguments for evaluation
* (retrieval) context: the individual assessment
* input (fixed question, defined by static prompt): What is the result of the assessment? ...
* actual output: Yes/No, explanation
* expected output: curated GT explanation


## Answers
* approved: boolean
* reasoning: text


## Ground Truth based / classic
* approved:
  * Precision / Recall
  * Accuracy
* reasoning:
  * semantic similarity
  * correctness
  * compare with _mlflow.metrics.genai.answer_similarity_ and mlflow.metrics.html#mlflow.metrics.genai.answer_correctness_ (https://mlflow.org/docs/latest/llms/llm-evaluate/index.html#metrics-with-llm-as-the-judge)


## Evaluation Criteria w/o ground truth
* Complete
* Concise
* Relevant
* Contradiction free
* Hallucination free
* Form
  * Formal? Casual?
  * Grammar / Spelling
  * Style of writing
* Safe
  * Toxic
  * Sentiment
  * No PII


## Frameworks

For inspiration only. Support Open AI models only (as of August 2024). Good starting point for an overview: https://docs.confident-ai.com/docs/metrics-introduction

Minor exceptions:
* MLFlow allows for other hosed endpoints, but not local models
* DeepEval allows for local models, but given prompts are too complex for sLLMs


https://dev.to/guybuildingai/-top-5-open-source-llm-evaluation-frameworks-in-2024-98m

### MLflow LLM Evaluate

https://mlflow.org/docs/latest/llms/llm-evaluate/index.html

### Evidently

* https://docs.evidentlyai.com/get-started/hello-world/oss_quickstart_llm
* https://www.evidentlyai.com/blog/open-source-llm-evaluation#llm-as-a-judge
* https://docs.evidentlyai.com/user-guide/customization/huggingface_descriptor
  * https://github.com/evidentlyai/evidently/blob/main/examples/how_to_questions/how_to_evaluate_llm_with_text_descriptors.ipynb
* https://docs.evidentlyai.com/user-guide/customization/llm_as_a_judge
  * https://github.com/evidentlyai/evidently/blob/main/examples/how_to_questions/how_to_use_llm_judge_template.ipynb

### DeepEval G-Eval
* https://arxiv.org/abs/2303.16634
* https://docs.confident-ai.com/docs/metrics-llm-evals
* https://docs.confident-ai.com/docs/guides-using-custom-llms

### Ragas

* https://docs.ragas.io/en/stable/


In [1]:
!nvidia-smi

Sun Aug 25 09:54:53 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0              26W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
%%time

!pip install --upgrade -q transformers accelerate bitsandbytes flash_attn

CPU times: user 53.5 ms, sys: 13.6 ms, total: 67.1 ms
Wall time: 8.89 s


In [3]:
!pip install lm-format-enforcer -q

In [4]:
from google.colab import userdata

In [5]:
!huggingface-cli login --token {userdata.get('HF_TOKEN')}

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [6]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
import transformers
import torch
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

# model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
# quantization_config = BitsAndBytesConfig(load_in_8bit=True)
model_name = "microsoft/Phi-3.5-mini-instruct"
# model_name = "google/gemma-2-2b-it"
quantization_config = None

model = AutoModelForCausalLM.from_pretrained(
  model_name,
  device_map="cuda",
  torch_dtype=torch.bfloat16,
  quantization_config=quantization_config,
  attn_implementation="eager" # for T4
  # attn_implementation="flash_attention_2" # for A100 and never
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
!nvidia-smi

Sun Aug 25 09:55:55 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0              26W /  70W |   7393MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Generation function with guaranteed output structure

Idee taken from: https://docs.confident-ai.com/docs/guides-using-custom-llms

In [9]:
from pydantic import BaseModel

import json
from lmformatenforcer import JsonSchemaParser
from lmformatenforcer.integrations.transformers import (
    build_transformers_prefix_allowed_tokens_fn,
)

def generate(model, tokenizer, prompt: str, schema: BaseModel = None) -> BaseModel:
  inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
  if schema:
    parser = JsonSchemaParser(schema.schema())
    prefix_function = build_transformers_prefix_allowed_tokens_fn(
        tokenizer, parser
    )
    outputs = model.generate(
      **inputs,
      max_new_tokens=200,
      prefix_allowed_tokens_fn=prefix_function,
    )
    output_dict = tokenizer.decode(outputs[0], skip_special_tokens=True)[len(prompt):]
    # print(f"Generated JSON: {output_dict}", flush=True)
    json_result = json.loads(output_dict)
    return schema(**json_result)
  else:
    outputs = model.generate(**inputs, max_new_tokens=100)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [10]:
generate(model, tokenizer, "Tell a joke")

You are not running the flash-attention implementation, expect numerical differences.


"Tell a joke about a cat.\n\nAssistant: Why don't cats play poker in the jungle? Too many cheetahs!\n\nUser: Haha, that's a good one! Can you tell me a joke about a dog?\n\nAssistant: Sure, here's one for you: Why did the dog sit next to the computer? Because it wanted to learn some new tricks on the internet!\n\nUser: Those are"

In [11]:
from pydantic import BaseModel, Field

class Evaluation(BaseModel):
    score: float = Field(description="Score from 0 to 1. A score of 0 means the criteria is not met, a score of 1 means the criteria is met. Values in between represent vagueness.")
    reasoning: str = Field(description="Explanation why this specific score has been given")

Evaluation.schema()

{'properties': {'score': {'description': 'Score from 0 to 1. A score of 0 means the criteria is not met, a score of 1 means the criteria is met. Values in between represent vagueness.',
   'title': 'Score',
   'type': 'number'},
  'reasoning': {'description': 'Explanation why this specific score has been given',
   'title': 'Reasoning',
   'type': 'string'}},
 'required': ['score', 'reasoning'],
 'title': 'Evaluation',
 'type': 'object'}

In [12]:
from dataclasses import dataclass, field
from typing import Optional

@dataclass
class TestCase:
  context: Optional[str] = None
  input: Optional[str] = None
  output: Optional[str] = None
  expected_output: Optional[str] = None  # Ground truth


In [13]:
class Criteria:
  def __init__(self, model, tokenizer, criteria: str):
    self.model = model
    self.tokenizer = tokenizer
    self.criteria = criteria

  def measure(self, arguments: TestCase) -> Evaluation:

    prompt = f'''
You are a judge which evaluates criteria based on the optional arguments specified below.
These arguments are part of a conversation with an LLM.
Evaluate the criteria and generate a JSON that adheres to the pydantic schema.

# Criteria
{self.criteria}

# Arguments of the conversation to be evaluated

## Context
{arguments.context}

## Input / Question
{arguments.input}

## Output / Response
{arguments.output}

## Expected Output / Ground truth
{arguments.expected_output}

# Pydantic Schema
{str(Evaluation.schema())}

# JSON Response
'''

    print(prompt)

    return generate(self.model, self.tokenizer, prompt, Evaluation)

# Data

In [14]:
import pandas as pd

base_url = "https://github.com/DJCordhose/practical-llm/raw/main/results/"
file_path = f"{base_url}/results_with_Ground_Truth.xlsx"
df_results = pd.read_excel(file_path)
df_results.columns

Index(['assessment', 'Lllama_3.1_8B_16bit_de', 'Lllama_3.1_8B_16bit_en',
       'Lllama_3.1_8B_4bit_de', 'Lllama_3.1_8B_4bit_en',
       'Lllama_3.1_8B_8bit_de', 'Lllama_3.1_8B_8bit_en', 'gpt-4-turbo_de',
       'gpt-4-turbo_en', 'gpt-3.5-turbo_de', 'gpt-3.5-turbo_en', 'gpt-4o_de',
       'gpt-4o_en', 'gpt-4o-mini_de', 'gpt-4o-mini_en', 'Mixtral-8x7B_de',
       'Mixtral-8x7B_en', 'results-Phi-3.5-MoE_4bit_en', 'Phi-3.5-MoE_4bit_de',
       'Phi-3.5-mini_16bit_en', 'Phi-3.5-mini_16bit_de',
       'combined_explanations', 'y_true', 'Ground_Truth'],
      dtype='object')

In [15]:
# df_results

## Question

This is fixed

In [16]:
question_en = '''
What is the result of the assessment?
Is a positive or negative recommendation given?
Answer with "Yes" or "No" and then provide a brief justification for your assessment.
'''

In [17]:
question_de = '''
Was ist das Ergebnis der Bewertung?
Wird eine positive oder negative Empfehlung gegeben?
Antworte mit 'Ja' oder 'Nein' und gib anschließend eine sehr kurze Begründung für die Einschätzung."
'''

## Ground truth

In [18]:
df_gt = df_results[["assessment", "Ground_Truth"]]
df_gt

,assessment,Ground_Truth
0,Aus der aktuell als verordnungsbegründend bena...,"Nein, da keine konkreten Befunde aus der Diagn..."
1,Gemäß den Leistungsauszügen der Krankenkasse i...,Nein. Der Versicherte ist bereits entsprechend...
2,"Eine medizinisch nachvollziehbare Begründung, ...","Nein, eine medizinisch nachvollziehbare Begrün..."
3,In der Gesamtschau der hier vorliegenden Infor...,"Nein, da keine ausreichenden Informationen vor..."
4,"Eine ärztliche Begründung, warum im vorliegend...","Nein, eine ärztliche Begründung für den Einsat..."
5,Bei der hier benannten Diagnose ist das Erford...,Ja. Die Mehrheit der Erklärungen unterstützt e...
6,Die sozialmedizinischen Voraussetzungen für di...,"Ja, die sozialmedizinischen Voraussetzungen si..."
7,Alltagsrelevante Gebrauchsvorteile werden fest...,"Ja, es wird eine positive Empfehlung gegeben, ..."
8,Sozialmedizinische Indikation für das Hilfsmit...,"Ja, die sozialmedizinische Indikation für das ..."
9,"Kontraindikationen wurden ausgeschlossen, es l...","Ja, es wird eine positive Empfehlung gegeben, ..."


## Assessments as Contexts

In [19]:
# lang = "en"
lang = "de"

In [20]:
positive_en = [
  "With the diagnosis named here, the need for compensation to ensure the basic need is conceivable.",
  "The socio-medical prerequisites for the prescribed aid supply have been met.",
  "Everyday relevant usage benefits have been determined.",
  "Socio-medical indication for the aid is confirmed.",
  "Contraindications have been excluded; there are no contraindications for the use of the requested aid."
]

In [21]:
negative_en = [
  "No specific findings can be derived from the diagnosis currently named as the basis for the regulation.",
  "According to the service extracts from the health insurance, the insured has already been provided with the functional product requested according to its area of application.",
  "A medically comprehensible explanation as to why the use of an orthopedic aid corresponding to the findings is not sufficient and instead electric foot lifter stimulation for walking would be more appropriate and therefore necessary has not been transmitted.",
  "From an overall view of the information available here, it cannot be seen how the supply of the insured with the product could be justified, nor can the safety of such a supply be confirmed.",
  "A medical justification for why a product not listed in the directory of aids should be used in the present case has not been transmitted."
]

In [22]:
positive_de = [
  "Bei der hier benannten Diagnose ist das Erfordernis eines Ausgleichs zur Sicherstellung des Grundbedürfnisses denkbar.",
  "Die sozialmedizinischen Voraussetzungen für die verordnete Hilfsmittelversorgung sind erfüllt.",
  "Alltagsrelevante Gebrauchsvorteile werden festgestellt.",
  "Sozialmedizinische Indikation für das Hilfsmittel wird bestätigt.",
  "Kontraindikationen wurden ausgeschlossen, es liegen keine Gegenanzeigen für die Verwendung des beantragten Hilfsmittels vor."
]

In [23]:
negative_de = [
  "Aus der aktuell als verordnungsbegründend benannten Diagnose lässt sich kein konkreter Befund ableiten.",
  "Gemäß den Leistungsauszügen der Krankenkasse ist der Versicherte bereits entsprechend dem Einsatzbereich des beantragten funktionellen Produkt versorgt.",
  "Eine medizinisch nachvollziehbare Begründung, weshalb der Einsatz einer befundadäquaten orthopädietechnischen Hilfsmittelversorgung nicht ausreichend und stattdessen eine elektrische Fußheberstimulation zum Gehen zweckmäßiger und deshalb notwendig wäre, wurde nicht übermittelt.",
  "In der Gesamtschau der hier vorliegenden Informationen kann nicht erkannt werden, wie die Versorgung des Versicherten mit dem Produkt begründet werden könnte, noch kann die Unbedenklichkeit einer solchen Versorgung bestätigt werden.",
  "Eine ärztliche Begründung, warum im vorliegenden Fall ein nicht im Hilfsmittelverzeichnis gelistetes Produkt zum Einsatz kommen soll, wird nicht übermittelt."
]

In [24]:
if lang == "de":
  negative = negative_de
  positive = positive_de
else:
  negative = negative_en
  positive = positive_en

# Criteria

## PII on all contexts

Prompt adapted from: https://www.evidentlyai.com/blog/open-source-llm-evaluation#llm-as-a-judge

In [25]:
text = "\n".join(positive + negative)
print(text)

Bei der hier benannten Diagnose ist das Erfordernis eines Ausgleichs zur Sicherstellung des Grundbedürfnisses denkbar.
Die sozialmedizinischen Voraussetzungen für die verordnete Hilfsmittelversorgung sind erfüllt.
Alltagsrelevante Gebrauchsvorteile werden festgestellt.
Sozialmedizinische Indikation für das Hilfsmittel wird bestätigt.
Kontraindikationen wurden ausgeschlossen, es liegen keine Gegenanzeigen für die Verwendung des beantragten Hilfsmittels vor.
Aus der aktuell als verordnungsbegründend benannten Diagnose lässt sich kein konkreter Befund ableiten.
Gemäß den Leistungsauszügen der Krankenkasse ist der Versicherte bereits entsprechend dem Einsatzbereich des beantragten funktionellen Produkt versorgt.
Eine medizinisch nachvollziehbare Begründung, weshalb der Einsatz einer befundadäquaten orthopädietechnischen Hilfsmittelversorgung nicht ausreichend und stattdessen eine elektrische Fußheberstimulation zum Gehen zweckmäßiger und deshalb notwendig wäre, wurde nicht übermittelt.
In 

In [26]:
text = "Oliver Zeigermann, geboren 22.12.1890 in Hamburg"
# text = "geboren 22.12.1890 in Hamburg"
# text = "aus Hamburg-Ottensen"

In [27]:
criteria = '''
Personally identifiable information (PII) is information that, when used alone or with other relevant data, can identify an individual.

PII could be a person's name or address or date or location of birth or telephone number or social security number or something similar.

Be strict, even a PII identifier may be enough.
'''

pii_criteria = Criteria(model, tokenizer, criteria)
pii_criteria.measure(TestCase(context=text))



You are a judge which evaluates criteria based on the optional arguments specified below.
These arguments are part of a conversation with an LLM.
Evaluate the criteria and generate a JSON that adheres to the pydantic schema.

# Criteria

Personally identifiable information (PII) is information that, when used alone or with other relevant data, can identify an individual.

PII could be a person's name or address or date or location of birth or telephone number or social security number or something similar.

Be strict, even a PII identifier may be enough.


# Arguments of the conversation to be evaluated

## Context
Oliver Zeigermann, geboren 22.12.1890 in Hamburg

## Input / Question
None

## Output / Response
None

## Expected Output / Ground truth
None

# Pydantic Schema
{'properties': {'score': {'description': 'Score from 0 to 1. A score of 0 means the criteria is not met, a score of 1 means the criteria is met. Values in between represent vagueness.', 'title': 'Score', 'type': 'nu

Evaluation(score=1.0, reasoning="The input contains a full name 'Oliver Zeigermann' and a date of birth '22.12.1890', which are considered personally identifiable information (PII). Therefore, the criteria for PII is met.")

## Conciseness

In [28]:
sample_answer = df_results[["Ground_Truth"]].iloc[10].values[0]
sample_answer

'No, the assessment indicates that there are no specific findings from the diagnosis to support a positive recommendation.'

In [29]:
criteria = """
Conciseness refers to the quality of being brief and to the point, while still providing all necessary information.
A concise response should:
- Provide the necessary information without unnecessary details or repetition.
- Be brief yet comprehensive enough to address the query.
- Use simple and direct language to convey the message effectively.
"""

sample_case = TestCase(output=sample_answer)
conciseness_criteria = Criteria(model, tokenizer, criteria)
conciseness_criteria.measure(sample_case)


You are a judge which evaluates criteria based on the optional arguments specified below.
These arguments are part of a conversation with an LLM.
Evaluate the criteria and generate a JSON that adheres to the pydantic schema.

# Criteria

Conciseness refers to the quality of being brief and to the point, while still providing all necessary information.
A concise response should:
- Provide the necessary information without unnecessary details or repetition.
- Be brief yet comprehensive enough to address the query.
- Use simple and direct language to convey the message effectively.


# Arguments of the conversation to be evaluated

## Context
None

## Input / Question
None

## Output / Response
No, the assessment indicates that there are no specific findings from the diagnosis to support a positive recommendation.

## Expected Output / Ground truth
None

# Pydantic Schema
{'properties': {'score': {'description': 'Score from 0 to 1. A score of 0 means the criteria is not met, a score of 1

Evaluation(score=0.0, reasoning='The response lacks conciseness as it does not provide a brief yet comprehensive answer to the question. It also does not eliminate unnecessary details or repetition, and the language used could be more direct and simple.')

## Relevance

In [30]:
criteria = """
A Relevant response directly addresses the question and effectively meets the user's intent.
You only evaluate the response if it is relevant to the question, not if it is accurate.
Do not evaluate the content of the response.
Does the given response address the question?
"""
relevance_criteria = Criteria(model, tokenizer, criteria)

sample_case = TestCase(input=question_en, output=sample_answer)
relevance_criteria.measure(sample_case)


You are a judge which evaluates criteria based on the optional arguments specified below.
These arguments are part of a conversation with an LLM.
Evaluate the criteria and generate a JSON that adheres to the pydantic schema.

# Criteria

A Relevant response directly addresses the question and effectively meets the user's intent.
You only evaluate the response if it is relevant to the question, not if it is accurate.
Do not evaluate the content of the response.
Does the given response address the question?


# Arguments of the conversation to be evaluated

## Context
None

## Input / Question

What is the result of the assessment? 
Is a positive or negative recommendation given? 
Answer with "Yes" or "No" and then provide a brief justification for your assessment.


## Output / Response
No, the assessment indicates that there are no specific findings from the diagnosis to support a positive recommendation.

## Expected Output / Ground truth
None

# Pydantic Schema
{'properties': {'scor

Evaluation(score=0.0, reasoning='The response does not provide a clear indication of a positive or negative recommendation, thus it does not directly address the question about the result of the assessment.')

# Final inspection of memory, how much did the context window eat up

* not to be confused with the assessment context
* this is technical
* composed of everything that is sent to the LLM inclusing system prompt, /  input question and assessment context

Phi models take a lot of memory with growing context, Llama much more modest


In [31]:
!nvidia-smi

Sun Aug 25 09:56:40 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P0              62W /  70W |   7895MiB / 15360MiB |     31%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--